In [16]:
import csv
import json
from datetime import datetime, timedelta
import pandas as pd

fn3 = 'recordScreenViews.csv'
f2 = open(fn3,"r")
filereader = csv.reader(f2)


mhist ={}
shist ={}

for line in filereader:
    timestamp = line[6]
    User_Type = line[8]
    Ip_Address = line [1]
    Screen_view = line [5]
    userId = line[7]
    #print (time)
    if 'company' in User_Type.lower():
        k = userId + '_' + Ip_Address + '_' + User_Type
        d = {}
        d['userType'] = User_Type
        d['ip']= Ip_Address
        d['screen'] = Screen_view
        d['timestamp'] = timestamp
        d['userId'] = userId
        if k in mhist.keys():
            mhist[k].append (d)
        else:
            mhist[k] = [d]
            
    elif 'student' in User_Type.lower():
        k = userId + '_' + Ip_Address + '_' + User_Type
        d = {}
        d['userType'] = User_Type
        d['ip']= Ip_Address
        d['screen'] = Screen_view
        d['timestamp'] = timestamp
        d['userId'] = userId
        if k in shist.keys():
            shist[k].append (d)
        else:
            shist[k] = [d]

#print (mhist)
#print (shist)
            

In [4]:

import time
from datetime import datetime, date, time, timedelta
from datetime import datetime as dt
import numpy as np

fmt = '%Y-%m-%d %H:%M:%S'

list1_session_company,list1_session1_company,list1_delta_company,list1_delta1_company = [],[],[],[]
list1_session_student,list1_session1_student,list1_delta_student,list1_delta1_student = [],[],[],[]

list1_session_word, list1_delta_word = [],[]
list2_company, list2_student = [], []


def data_processing (dictionary, word):
    list1_session_word = []
    list1_delta_word = []
    for key,values in dictionary.items():
        df = pd.DataFrame(values).sort_values("timestamp")
        df['timestamp'] = pd.to_datetime (df['timestamp']) 
        df['delta'] = (df['timestamp']-df['timestamp'].shift())    
        df['time'] = df['delta'].dt.total_seconds()#.dropna()
        df['More_Less_300_Seconds'] =  df['time'] > 300
        df['session'] = (df.More_Less_300_Seconds.cumsum() + 1).map(str).shift()
        df['session'] = df['session'].fillna(method='bfill')
        #print (df)
        for item in df['session']:
            list1_session_word.append (item)
        for item1 in df['delta']:
            list1_delta_word.append (str(item1))
    return list1_session_word, list1_delta_word

list1_session_company,list1_delta_company = data_processing (mhist,'company')
list1_session_Student,list1_delta_Student = data_processing (shist,'student')

def data_preparation (TheList):
    i = -1
    list1_both = []
    for value in TheList:
        i+=1
        if i >=1: 
            list1_both.append(TheList[i])
    list1_both.append('nan')
    return list1_both

list1_session1_company = data_preparation (list1_session_company)
list1_delta1_company = data_preparation (list1_delta_company)
list1_session1_student = data_preparation (list1_session_Student)
list1_delta1_student = data_preparation (list1_delta_Student)


def data_processing1 (dict):
    list2 = []
    for item in dict.values():
        for element in item:
            list2.append (element)

    df_list2 = pd.DataFrame(list2)
    return df_list2

df_list2_company = data_processing1 (mhist)
df_list2_student = data_processing1 (shist)

df_list2_company_student= [df_list2_company, df_list2_student]
df_list2_company_student = pd.concat(df_list2_company_student)
    
# Convert the list1_session1_company,list1_delta1_company, list1_session1_student and 
# list1_delta1_student to dataframe

df_session1_company = pd.DataFrame(list1_session1_company)
df_session1_student = pd.DataFrame(list1_session1_student)
df_session_company_student= [df_session1_company, df_session1_student]
df_session_company_student = pd.concat(df_session_company_student, names=['session'])

df_delta1_company = pd.DataFrame(list1_delta1_company)
df_delta1_student = pd.DataFrame(list1_delta1_student)
df_delta_company_student= [df_delta1_company, df_delta1_student]
df_delta_company_student = pd.concat(df_delta_company_student, names=['delta'])

#print (df_delta_company_student)

# Total Dataframe

Total_Dataframe_company_student = pd.concat([df_list2_company_student, df_session_company_student,df_delta_company_student], axis=1)
Total_Dataframe_company_student.columns = ['userType','ip','screen','timestamp','userId','session','delta']

#df.columns = [
# Convert the dataframe to csv file 
Total_Dataframe_company_student.to_csv('Update_ recordScreen_Views_sessions2.csv',index=False)



In [14]:
fn4 = 'Update_ recordScreen_Views_sessions2.csv'
f4 = open(fn4,"r")
filereader1 = csv.reader(f4)

mhist_session ={}
shist_session ={}

for line in filereader1:
    session = line [5]
    timestamp = line[3]
    User_Type = line[0]
    Ip_Address = line [1]
    Screen_view = line [2]
    userId = line[4]
    delta = line [6]
    #print (time)
    if 'company' in User_Type.lower():
        k = User_Type + '_' + userId + '_' + Ip_Address + '_' + session  
        d = {}
        d['timestamp'] = timestamp
        d['screen'] = Screen_view
        d['elapsedTime'] = delta
        if k in mhist_session.keys():
            mhist_session [k].append (d)
        else:
            mhist_session [k] = [d]
            
    elif 'student' in User_Type.lower():
        k = User_Type + '_' + userId + '_' + Ip_Address + '_' + session 
        d = {}
        d['timestamp'] = timestamp
        d['screen'] = Screen_view
        d['elapsedTime'] = delta
        if k in shist_session.keys():
            shist_session[k].append (d)
        else:
            shist_session[k] = [d]

list_output_mhist= []
list_output_shist = []

for k, values in mhist_session.items():
    d = {}
    data = k.split('_')
    d['user_Type'] = data[0]
    d['user_id'] = data[1]
    d['ip'] = data[2]
    d['session'] = data[3]
    d['screens'] = values
    list_output_mhist.append(d)
    
for k, values in shist_session.items():
    d = {}
    data = k.split('_')
    d['user_Type'] = data[0]
    d['user_id'] = data[1]
    d['ip'] = data[2]
    d['session'] = data[3]
    d['screens'] = values
    list_output_shist.append(d)

json_mhist_session1 = json.dumps(list_output_mhist)
json_shist_session1 = json.dumps(list_output_shist)

#print(json_mhist_session1)
print (json_shist_session1)


[{"user_Type": "student", "user_id": "00w66n6yCQT6ljA49XmpTwF8cdH3", "ip": "192.168.86.57", "session": "1", "screens": [{"timestamp": "2020-11-15 05:58:20", "screen": "CreateStudentProfile", "elapsedTime": "NaT"}]}, {"user_Type": "student", "user_id": "0CiLaaQ2x0bLOgZjd2ubgnkdf4G3", "ip": "10.0.0.221", "session": "1", "screens": [{"timestamp": "2020-11-09 20:58:22", "screen": "CreateStudentProfile", "elapsedTime": "0 days 00:04:16"}, {"timestamp": "2020-11-09 21:02:38", "screen": "StudentProfile", "elapsedTime": "0 days 00:00:30"}, {"timestamp": "2020-11-09 21:03:08", "screen": "Newsfeed", "elapsedTime": "0 days 00:00:10"}]}, {"user_Type": "student", "user_id": "0CiLaaQ2x0bLOgZjd2ubgnkdf4G3", "ip": "10.0.0.221", "session": "", "screens": [{"timestamp": "2020-11-09 21:03:18", "screen": "Menu", "elapsedTime": "NaT"}]}, {"user_Type": "student", "user_id": "0pt491pQgAUnooqCeGCDndza7Zs2", "ip": "192.168.0.202", "session": "1", "screens": [{"timestamp": "2020-12-18 17:57:00", "screen": "Crea